Topic entitlement works much better.

In [ ]:
%tensorflow_version 2.x

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 4.2 MB/s 
     |████████████████████████████████| 56 kB 3.6 MB/s 
     |████████████████████████████████| 3.3 MB 40.4 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 


In [ ]:
from transformers import pipeline

In [ ]:
import os
import datetime
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
part_length=1024
half_part_length = 512
max_topic_score=0.95

In [ ]:
Data = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Data/'


ps_filename='philologist_samples.csv'
ps_full_filename=os.path.join(Data, ps_filename)

In [ ]:
psMessages = pd.read_csv(ps_full_filename, error_bad_lines=False, index_col=False)

In [ ]:
model = 'cointegrated/rubert-base-cased-nli-threeway'

In [ ]:
classifier = pipeline('zero-shot-classification', model=model)

Downloading:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
psMessages['topics']=''
psMessages['topics_num']=0

In [ ]:
candidate_labels = [
'бизнес',
'крутится',
'иммиграция',
'филология',
'грамматика',
'кафедра',
'аспирантура',
'университет',
'студент',
'америка',
'канада',
'почасовик',
'учитель',
'Учеба',
'вуз'
]

In [ ]:
#set labels
for index,row in psMessages.iterrows():
    text=row['Sample']
    lt = len(text)
    parts_labels=list()
    if lt>part_length:
        num_parts = int(lt/1024)
        reminder = lt % 1024
        for i in range(0,num_parts,1):        
            Start=i*part_length
            End=(i+1)*part_length
            #print('.', end = ' '),
            labels=classifier(text[Start:End], candidate_labels, multi_label=True)
            parts_labels.append(labels)
            Start=half_part_length+i*part_length
            End=half_part_length+((i+1)*part_length)
            #print('.', end = ' '),
            labels=classifier(text[Start:End], candidate_labels, multi_label=True)  
            parts_labels.append(labels)
        if reminder > 0:
            Start=lt - part_length
            End=lt    
            #print('.', end = ' '),
            labels=classifier(text[Start:End], candidate_labels, multi_label=True)
    else:
        #print('.', end = ' '),
        labels=classifier(text, candidate_labels, multi_label=True)
        parts_labels.append(labels)
    #pieces labels
    d = dict()
    for cl in candidate_labels:
        d[cl]=[]
    for pl in parts_labels:
        ld=list()
        for l in zip(pl['labels'],pl['scores']):
            d[l[0]].append(l[1])
    #max weight each label from pieces        
    labels=dict()
    for cl in candidate_labels:
        labels[cl]=max(d[cl])
    #final topics with weight > max_topic_score
    #topics = [k for k,v in labels.items() if v>max_topic_score]
    #topics.sort()
    df =pd.DataFrame(labels.items())
    df.columns=['topic','score']
    df.sort_values(['score'],ascending=False, inplace=True)
    #print(df)
    #final topics with weight > max_topic_score
    #topics = [k for k,v in labels.items() if v>max_topic_score]
    #topics.sort()
    topics=df[df['score']>max_topic_score]['topic'].tolist()
    psMessages.at[index,'topics']=','.join(topics)
    psMessages.at[index,'topics_num']=len(topics)
    print('.', end = ' '),
psMessages.to_csv(ps_full_filename, header=True, index=False)

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
psMessages.head()

,Sample_Id,Sample,headline_ria,sum_gazeta,long_sum_gazeta,topics,topics_num
0,84722252,"Сначала хотела не читать ЗЫ, но первая же фраз...",""" я только виноват в труднои истории с географ...",В преддверии нового учебного года я решила под...,В преддверии нового учебного года я хочу подел...,"филология,иммиграция",2
1,97328387,"А Вы бы не работали по 8 часов, а защитили бы ...",""" я живу, как ежикии городиш ""","На Западе все живут там, где хотят, где им дал...","Столкнуть лбами конкурентов и всех вылизать, о...","крутится,аспирантура,кафедра,филология,универс...",5
2,97321610,"Ой, ну такое состояние общества. Как один раз ...",""" я не знал, что мы не должны думать о том, чт...",В современном мире быдло никуда не исчезнет. О...,Быдло никуда не исчезнет. Оно еще впереди Ваше...,"иммиграция,кафедра,крутится",3
3,97358217,"Да, я образец шикарных генов, рожденная породи...",""" маман мои с горшками ""","В реальном мире люди пристраиваются не потому,...","В реальном мире люди пристраиваются не потому,...","иммиграция,аспирантура,университет,филология,к...",8
4,97497137,"Ваши сельские уже по пять паспортов у каждой, ...","украина, канада и канада : от бабушек до бабушек","Российские бабушки, живущие в сельских селах, ...","Не знаю, как Вы, но я не беру украинских уборщ...","иммиграция,канада,филология,грамматика",4
